In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('Bank.csv') # 분석할 데이터 불러오기

In [3]:
X = df.iloc[:, :-1] # 독립변수 matrix

In [4]:
y = df.iloc[:, -1] # 종속변수 matrix

In [14]:
y = y.replace('no', 0) # 종속변수의 no를 0으로 변환
y = y.replace('yes', 1) # 종속변수의 yes를 1로 변환

In [52]:
# 독립변수가 연속형인 데이터
X_num = df.iloc[:, [0, 5, 9, 11, 12, 13, 14]]
# 독립변수가 이산형인 데이터
X_cat = df.iloc[:, [1, 2, 3, 4, 6, 7, 8, 10, 15]]

In [5]:
# 1번째 독립변수의 최소값, 최대값, 평균값 확인
""""""np.min(X.iloc[:, 0]), np.max(X.iloc[:, 0]), np.mean(X.iloc[:, 0])
# 6번째 독립변수의 최소값, 최대값, 평균값 확인
""""""np.min(X.iloc[:, 5]), np.max(X.iloc[:, 5]), np.mean(X.iloc[:, 5])
# 10번째 독립변수의 최소값, 최대값, 평균값 확인
""""""np.min(X.iloc[:, 9]), np.max(X.iloc[:, 9]), np.mean(X.iloc[:, 9])

(19, 87, 41.11130939103279)

(-3313, 71188, 1421.987731429846)

(1, 31, 15.91590452821771)

In [7]:
# 1번째 독립변수를 35세 미만은 under35
# 35세 이상 50세 미만은 under50
# 50세 이상은 upper50으로 이산화 시키기
under35_idx = np.where(X.iloc[:, 0] < 35)[0]
under50_idx = np.where((35 <= X.iloc[:, 0]) & (X.iloc[:, 0] < 50))[0]
upper50_idx = np.where(50 <= X.iloc[:, 0])[0]

X.iloc[under35_idx, 0] = 'under35'
X.iloc[under50_idx, 0] = 'under50'
X.iloc[upper50_idx, 0] = 'upper50'

In [8]:
# 이산화 완료된 1번째 변수의 value 별 개수 확인
sum(X.iloc[:, 0] == 'under35'), sum(X.iloc[:, 0] == 'under50'), sum(X.iloc[:, 0] == 'upper50')

(1468, 2006, 1009)

In [9]:
# 6번째 독립변수를 0원 미만은 negative
# 0원 이상 5000원 미만은 normal
# 5000원 이상은 wealthy로 이산화 시키기
""""""negative_idx = np.where(X.iloc[:, 5] < 0)[0]
normal_idx = np.where((0 <= X.iloc[:, 5]) & (X.iloc[:, 5] < 5000))[0]
wealthy_idx = np.where(5000 <= X.iloc[:, 5])[0]

X.iloc[negative_idx, 5] = 'negative'
X.iloc[normal_idx, 5] = 'normal'
X.iloc[wealthy_idx, 5] = 'wealthy'

In [10]:
# 이산화 완료된 6번째 변수의 value 별 개수 확인
sum(X.iloc[:, 5] == 'negative'), sum(X.iloc[:, 5] == 'normal'), sum(X.iloc[:, 5] == 'wealthy')

(366, 3811, 306)

In [11]:
# 10번째 독립변수를 15일 이하는 first_half
# 15일 초과는 latter_half로 이산화 시키기
""""""first_idx = np.where(X.iloc[:, 9] <= 15)[0]
latter_idx = np.where(15 < X.iloc[:, 9])[0]

X.iloc[first_idx, 9] = 'first_half'
X.iloc[latter_idx, 9] = 'latter_half'

In [12]:
# 이산화 완료된 10번째 변수의 value 별 개수 확인
""""""sum(X.iloc[:, 9] == 'first_half'), sum(X.iloc[:, 9] == 'latter_half')

(2154, 2329)

In [13]:
# 독립변수가 연속형인 데이터
""""""X_num = X.iloc[:, [11, 12, 13, 14]]
# 독립변수가 이산형인 데이터
""""""X_cat = X.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15]]

In [15]:
tmp_df = X_cat # 이산형 데이터는 임시로 다른 곳에 저장
final_X = X_num # 연속형 데이터는 dummy 변수가 필요 없으므로 분석할 데이터에 저장

# 이산형 데이터를 하나씩 dummy 변수로 변환하는 작업 수행
for i in X_cat.columns:
    cat_list = pd.get_dummies(tmp_df[i], drop_first = True, prefix = i)
    final_X = final_X.join(cat_list) # dummy 변수로 만든 이산형 데이터를 분석용 데이터에 합치기

In [16]:
# Logistic regression 함수 import
from sklearn.linear_model import LogisticRegression
# k-fold cross-validation 함수 import
from sklearn.model_selection import KFold

logit_reg = LogisticRegression() # Logistic regression 함수 명명

In [17]:
# scaling 함수 import
from sklearn.preprocessing import StandardScaler

In [18]:
n_iter = 10 # 전체 실험 반복 횟수

iter_train_result = [] # training set의 전체 값 초기화
iter_test_result = [] # test set의 전체 값 초기화

for epoch in np.arange(n_iter):
    
    cv = KFold(n_splits = 5, shuffle = True) # 5-fold cv 함수 입력
    
    fold_train_result = 0 # training set의 5-fold R-squared 값 초기화
    fold_test_result = 0 # test set의 5-fold R-squared 값 초기화
    
    for tr_idx, te_idx in cv.split(final_X):
        
        X_train = final_X.iloc[tr_idx] # cross-validation index를 통한 독립변수 training set
        X_test = final_X.iloc[te_idx] # cross-validation index를 통한 독립변수 test set
        y_train = y[tr_idx] # cross-validation index를 통한 종속변수 training set
        y_test = y[te_idx] # cross-validation index를 통한 종속변수 test set
                
        scaler = StandardScaler() # scaler 함수 명명
        scaler = scaler.fit(X_train.iloc[:, 0:np.shape(X_num)[1]]) # scaler 함수 fitting
        # training set 내 연속형 데이터의 scaling
        zs_X_train = scaler.transform(X_train.iloc[:, 0:np.shape(X_num)[1]])
        # training set의 연속형과 이산형 데이터 합치기
        zs_X_train = np.c_[zs_X_train, X_train.iloc[:, np.shape(X_num)[1]:].values]
        
        """"""logit_reg = logit_reg.fit(zs_X_train, y_train) # Logistic regression 예측 모형 학습
        
        # test set 내 연속형 데이터의 scaling (training set의 정보 이용)
        zs_X_test = scaler.transform(X_test.iloc[:, 0:np.shape(X_num)[1]])
        # test set의 연속형과 이산형 데이터 합치기
        zs_X_test = np.c_[zs_X_test, X_test.iloc[:, np.shape(X_num)[1]:].values]
        
        # 5-fold의 training set 학습 결과 저장
        fold_train_result += logit_reg.score(zs_X_train, y_train)
        # 5-fold의 test set 학습 결과 저장
        fold_test_result += logit_reg.score(zs_X_test, y_test)
    
    # 5-fold의 training set 학습 결과의 평균
    fold_train_result = fold_train_result / cv.get_n_splits(final_X)
    # 5-fold의 test set 학습 결과의 평균
    fold_test_result = fold_test_result / cv.get_n_splits(final_X)
    
    iter_train_result = iter_train_result + [fold_train_result] # training set의 전체 R-squared 값 저장
    iter_test_result = iter_test_result + [fold_test_result] # test set의 전체 R-squared 값 저장

/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-package

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/home/pirl/anaconda3/lib/python3.7/site-package

In [19]:
# training set의 logistic regression 결과값 (평균값 & 표준편차)
""""""train_mean = np.mean(iter_train_result)
train_mean
""""""train_std = np.std(iter_train_result)
train_std

0.9025930814487652

0.0003456068027903152

In [20]:
# test set의 logistic regression 결과값 (평균값 & 표준편차)
""""""test_mean = np.mean(iter_test_result)
test_mean
""""""test_std = np.std(iter_test_result)
test_std

0.9002002707437489

0.001138671805609043

In [21]:
# Logistic regression 함수 import
from sklearn.linear_model import LogisticRegression
logit_reg_cf = LogisticRegression() # Logistic regression 함수 명명

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # scaler 함수 명명
scaler = scaler.fit(final_X.iloc[:, 0:np.shape(X_num)[1]]) # scaler 함수 fitting

/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [23]:
# 전체데이터 내 연속형 데이터의 scaling
zs_final_X = scaler.transform(final_X.iloc[:, 0:np.shape(X_num)[1]])
# 전체데이터의 연속형과 이산형 데이터 합치기
zs_final_X = np.c_[zs_final_X, final_X.iloc[:, np.shape(X_num)[1]:].values]

/home/pirl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


In [24]:
# 전체데이터를 통한 logistic regression 예측 모형 학습
""""""logit_reg_cf.fit(zs_final_X, y)

/home/pirl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [25]:
# transform to original values
std_coef = np.r_[scaler.var_ ** 0.5, np.ones(np.shape(final_X)[1] - np.shape(X_num)[1])]

In [26]:
# 원래의 변수에 대한 회귀 계수들의 승수비 (odds ratio)
""""""odds_ratio = np.exp(logit_reg_cf.coef_ / std_coef)

In [27]:
# 승수비 결과를 pandas의 data frame으로 정리
""""""Final_Results = pd.DataFrame(odds_ratio, columns = final_X.columns)
Final_Results

,duration,campaign,pdays,previous,age_under50,age_upper50,job_blue-collar,job_retired,job_self-employed,job_services,...,balance_wealthy,housing_yes,loan_yes,contact_telephone,contact_unknown,day_latter_half,month_latter_half,poutcome_other,poutcome_success,poutcome_unknown
0,1.004011,0.940539,0.999095,0.994475,0.926856,0.88117,0.656082,1.947277,0.780598,0.825015,...,0.787579,0.578756,0.482985,0.94105,0.304586,1.024534,0.763071,1.404451,9.642006,0.625558
